In [1]:
import galprime as gp

In [14]:
test_fn = "gprime_out/tempfiles/11081444_0_0_0_0_done.pkl"

obj = gp.load_object(test_fn)



In [15]:
print(obj.stop_code)
print(obj.logger)


10
<Logger RUN_11081444 (WARNING)>
